In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [ ]:
kernel = sk.Kernel()

deployment = 'deploymentname'
api_key = 'apikey'
endpoint = 'openaiURL'

kernel.add_chat_service("chat", AzureChatCompletion(deployment, endpoint, api_key))

base_skills_directory = '/lakehouse/default/Files/Skills'

skills = {
**kernel.import_semantic_skill_from_directory(base_skills_directory , "dataskill"),
}

csv_skill = skills["csv"]
pysql_skill = skills["spark"]

In [ ]:
import pandas as pd
df = pd.read_csv("/lakehouse/default/Files/csv/ProductList.csv")

In [ ]:
panda_context_variables = sk.ContextVariables(variables={
"question": "Generate a bar chart to list Product Type and display the number of products owned by the corresponding Product Type"
})
panda_result = await csv_skill.invoke_async(variables=panda_context_variables )
pandasql = panda_result .result.replace("\\n","").replace("\\n\\n", "")
exec(pandasql)


In [ ]:
print(pandasql)

In [ ]:
sql_skill = skills["onyxall"]

sql_context_variables = sk.ContextVariables(variables={
"datasource": "geolake",
"question": "list the total of metropolitan regions by country"
})
sql_result = await sql_skill.invoke_async(variables=sql_context_variables)
sql_result.result.replace("\n"," ")
print(sql_result)

In [ ]:
sql_skill = skills["onyxall"]

sql_context_variables = sk.ContextVariables(variables={
"datasource": "geolake",
"question": "Could you list the metropolitan regions in Deutschland and their city name?"
})
sql_result = await sql_skill.invoke_async(variables=sql_context_variables)
sql_result.result.replace("\n"," ")
print(sql_result)

In [ ]:
sql_skill = skills["onyxall"]

sql_context_variables = sk.ContextVariables(variables={
"datasource": "geolake",
"question": "Could you provide the total of metropolitan regions by city in Deutschland?"
})
sql_result = await sql_skill.invoke_async(variables=sql_context_variables)
sql_result.result.replace("\n"," ")
print(sql_result)

In [ ]:
%%sql
SELECT m.CityName, COUNT(*) AS TotalMetropolitanRegions
FROM Metropolitan m
JOIN SpecificRegions sr ON m.RegionCode = sr.RegionCode
JOIN BasicRegions br ON sr.BasicRegionCode = br.RegionCode
JOIN MajorRegions mr ON br.MajorRegionCode = mr.RegionCode
JOIN Countries c ON mr.CountryCode = c.CountryCode
WHERE c.Country = 'Deutschland'
GROUP BY m.CityName
ORDER BY TotalMetropolitanRegions DESC;